# SVM woe


## Adatok betöltése

In [92]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bc050f45180b4c1bb8886542b85a43fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6akYbYHItNWmuiZPpgS8f0DnZNTVPSN3hYgF_EVLfHhd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_input = pd.read_csv(body, sep=';',decimal=',')
train_input.head(10)


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,-0.893508
2,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,-0.224232
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.224232
5,0,-0.029802,-0.180531,-0.210588,-0.365448,-0.883030,0.585450
6,0,0.276928,-0.180531,0.728017,0.603264,0.232891,0.585450
7,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
8,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
9,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,0.585450


In [93]:

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_input = pd.read_csv(body, sep=';',decimal=',')
test_input.head()


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,0.585450
2,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.893508


In [94]:
train_input.shape

(7339, 7)

In [95]:
test_input.shape

(3083, 7)

## Adatok előkészítése

In [96]:
import numpy as np

In [97]:
X_train = np.asarray(train_input[['customer_type', 'product_type', 'onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier']])
X_train[0:1]

array([[-0.02980206, -0.18053093,  0.01095466,  0.6032641 ,  0.23289082,
         0.58545026]])

In [98]:
X_test = np.asarray(test_input[['customer_type', 'product_type', 'onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier']])
X_test[0:1]

array([[-0.02980206, -0.18053093,  0.01095466,  0.6032641 ,  0.23289082,
         0.58545026]])

In [99]:
y_train = np.asarray(train_input['DPD90_12m'])
y_train [0:5]

array([0, 0, 0, 0, 0])

In [100]:
y_test = np.asarray(test_input['DPD90_12m'])
y_test [0:5]

array([0, 0, 0, 0, 0])

## Modellezés

In [101]:
from sklearn.svm import SVC
LR = SVC(probability=True, kernel='sigmoid').fit(X_train,y_train)
LR

SVC(kernel='sigmoid', probability=True)

In [102]:
yhat_test = LR.predict_proba(X_test)
yhat_test

array([[0.98172898, 0.01827102],
       [0.90434771, 0.09565229],
       [0.97284435, 0.02715565],
       ...,
       [0.88521912, 0.11478088],
       [0.90807864, 0.09192136],
       [0.98187122, 0.01812878]])

In [103]:
yhat_train = LR.predict_proba(X_train)
yhat_train

array([[0.98172898, 0.01827102],
       [0.92243752, 0.07756248],
       [0.9699641 , 0.0300359 ],
       ...,
       [0.97978292, 0.02021708],
       [0.9701899 , 0.0298101 ],
       [0.97134871, 0.02865129]])

In [104]:
tech, y_predicted_train = zip(*yhat_train)
y_predicted_train[0:5]

(0.018271015135672187,
 0.07756247643573855,
 0.030035901061733757,
 0.017370636449471764,
 0.015066927275987973)

In [105]:
tech, y_predicted_test = zip(*yhat_test)
y_predicted_test[0:5]

(0.018271015135672187,
 0.0956522915862837,
 0.02715565151887103,
 0.02715565151887103,
 0.02342022818320937)

## Modell ereje

In [106]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_train, y_predicted_train)
gini=2*roc-1
gini

0.2727209965339723

In [107]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test, y_predicted_test)
gini=2*roc-1
gini

0.20478955508895602